https://tensorflow.google.cn/tutorials/eager/custom_training_walkthrough

In [16]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 1.13.1
Eager execution: True


In [11]:
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))

Local copy of the dataset file: /Users/luoyonggui/.keras/datasets/iris_training.csv


In [12]:
!head -n5 {train_dataset_fp}

120,4,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0


In [13]:
# column order in CSV file
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
Label: species


In [14]:
class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']

### 创建一个 tf.data.Dataset
TensorFlow 的 Dataset API 可处理在向模型加载数据时遇到的许多常见情况。这是一种高阶 API，用于读取数据并将其转换为可供训练使用的格式。

由于数据集是 CSV 格式的文本文件，请使用 make_csv_dataset 函数将数据解析为合适的格式。由于此函数为训练模型生成数据，默认行为是对数据进行随机处理 (shuffle=True, shuffle_buffer_size=10000)，并且无限期重复数据集 (num_epochs=None)。我们还设置了 batch_size 参数。



In [21]:
df = pd.read_csv(train_dataset_fp, skiprows=1, header=None)
df.columns = column_names
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [23]:
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((df.values[:, :4], df.values[:, 4]))

#### 启用 Eager Execution 后，这些 Dataset 对象便可迭代。我们来看看一批特征：

In [24]:
features, labels = next(iter(train_dataset))

In [25]:
features

<tf.Tensor: id=8, shape=(4,), dtype=float64, numpy=array([6.4, 2.8, 5.6, 2.2])>

In [26]:
train_dataset.batch(2)

<DatasetV1Adapter shapes: ((?, 4), (?,)), types: (tf.float64, tf.float64)>

In [27]:
features, labels = next(iter(train_dataset))
features

<tf.Tensor: id=19, shape=(4,), dtype=float64, numpy=array([6.4, 2.8, 5.6, 2.2])>

## 使用 Keras 创建模型

In [32]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])

In [35]:
predictions = model(train_dataset)

ValueError: Attempt to convert a value (<DatasetV1Adapter shapes: ((4,), ()), types: (tf.float64, tf.float64)>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>) to a Tensor.